# Cyklisté v Praze
V této úloze se budeme snažit vytvořit model toho, jak jezdí cyklisté kolem jedné kamery umístěné v [Praze v Bohnicích](https://mapy.cz/turisticka?x=14.3983934&y=50.1427415&z=16&source=coor&id=14.39908%2C50.14313).

Tato kamera poskytuje data, kdy došlo k průjezdu cyklistou
```python
from requests import request
import pandas as pd
headers = {
  'Content-Type': 'application/json; charset=utf-8',
  'x-access-token': '?????'
}

dfs = []
ranges = list(zip(
        list(pd.date_range("2019-01-01", "2020-12-31", freq="MS")),
        list(pd.date_range("2019-01-01", "2020-12-31", freq="M"))
))


for s, e in ranges: # přístup max 10 000 položek, rozdělíme do měsíců
    r = request("get", 'https://api.golemio.cz/v2/bicyclecounters/temperatures?id=camea-BC_ZA-KLBO&from={}Z&to={}'.format(s.isoformat(), e.isoformat()), headers=headers)
    print(r.text)
    dfs.append(pd.DataFrame(r.json()))

df_cam = pd.concat(dfs)

df_c = df_cam.set_index(pd.to_datetime(df_cam["measured_from"])).resample("d").agg({"value":"sum"}).tz_convert(None)
df_c.plot()
df_c.to_pickle("prg_cycl.pkl.gz")
```

Zdroj dat:

https://golemioapi.docs.apiary.io/#reference/traffic/bicyclecounters/get-bicyclecounters-detections

__Problémy__
* data jsou známá až od poloviny roku 2019
* CHMI poskytuje historická data počasí ročně :(

In [ ]:
###
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Načtení a vyčištění primárních dat

In [ ]:
###
# Načtení dat
df = pd.read_pickle("prg_cycl.pkl.gz")
df

In [ ]:
# a jejich zobrazení
df.plot() ###

Nyní musíme odstranit hodnoty, které jsou mimo rozsah (v našem případě pod hodnotu 500), protože se jedná zřejmě o chybu kamery

In [ ]:
dfd = df[df["value"] > 500].asfreq("d", method="ffill")
dfd.plot()


## Načtení dat - nezávislých proměnných
Načteme data z CHMU - pro stanici Praha-Ruzyně. Používáme techniky jako v přednášce _Pandas_.

In [ ]:
###
meteo = pd.read_excel("P1PRUZ01.xls", header=3, sheet_name=[
    "teplota průměrná", "úhrn srážek", "sluneční svit", "celková výška sněhu"
    ])
meteo

In [ ]:
###
# Převod na Pandas DataFrame
# pro každý sheet provedeme:
# - melt
# - vyparsujeme datum
# - spojíme data

df_meteo = None
for k in meteo:

    df_l = meteo[k].melt(["rok", "měsíc"], var_name="den", value_name=k).dropna()
    df_l["den"] = df_l["den"].str.replace(".", "")

    df_l["date"] = pd.to_datetime(df_l["rok"].astype(str) + "-" + df_l["měsíc"].astype(str) + "-" + df_l["den"].str.replace(".", ""))

    df_l = df_l[["date", k]].set_index("date")

    if df_meteo is None:
        df_meteo = df_l
    else:
        df_meteo = df_meteo.merge(df_l, left_index=True, right_index=True)


# Na závěr vybereme jen rok 2018 a 2019
df_meteo = df_meteo.loc["2018":"2019"].sort_index()
# a nepříjemným trikem posuneme rok 2018 na 2020 a tím doplníme chybějící data
# opravdu nepěkná operace -> lepší by bylo získat pravá data za rok 2020
df_meteo = pd.concat([df_meteo.loc["2019"], df_meteo.loc["2018"].shift(2*365, "d")])
df_meteo

## Spojení dat do jednoho dataframe

In [ ]:
###
# Prvně přidáme informaci o počasí
dfd = dfd[["value"]].merge(df_meteo, left_index=True, right_index=True)
dfd

In [ ]:
###
# Přidáme informaci o dnu v týdnu

dfd["dow"] = dfd.index.dayofweek

days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(7):
    dfd[days[i]] = (dfd["dow"] == i).astype(float)
dfd

## Hledání regresního modelu
První krok je zjistit, zda neexistuje korelace mezi našimi možnými nezávislými proměnnými

In [ ]:
sns.pairplot(dfd[["teplota průměrná", "úhrn srážek", "sluneční svit", "celková výška sněhu"]]) ###

Vidíme, že data nekorelují, takže můžeme vytvořit regresní model. Můžeme volit následující nezávislé proměnné
```python
ks = []
ks += ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'] 
ks += ["teplota průměrná"]
ks += ["úhrn srážek"]
ks += ["sluneční svit"]
ks += ["celková výška sněhu"]
```

In [ ]:
# vytvoříme pole příznaků
ks = []
ks += ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'] 
ks += ["teplota průměrná"]
ks += ["úhrn srážek"]
ks += ["sluneční svit"]
ks += ["celková výška sněhu"]

# vytvoříme pomocný dataframe pouze s rokem 2019
df_tmp = dfd.loc["2019"].copy() # todo pridat i rok 2020

# Vytvoříme vektory X a y
X = df_tmp[ks].astype("f")
y = df_tmp["value"]

# Spustíme trénování
model = LinearRegression(fit_intercept=True, normalize=True)
model.fit(X, y)

# Vypíšme kvalitu a koeficienty
print(model.score(X, y))

# Určíme predikovanou hodnotu
df_tmp["predicted"] = model.predict(X)


ax = df_tmp.plot(y="value", figsize=(12, 8))
df_tmp.plot(y="predicted", ax=ax)

Podíváme se na to, jak jednotlivé nezávislé proměnné ovlivňují výstup. Analyzujeme vektor $w$, který je uložený v `model.coef_`.

In [ ]:
print(list(zip(ks, model.coef_)))

In [ ]:
###
# zkusíme to aplikovat na data z roku 2020 (pozor, ty vlastně nemáme, takže z roku 2018)
# proto je chybovost ještě větší
df_tmp = dfd.loc["2020"].copy()
X = df_tmp[ks]

df_tmp["predicted"] = model.predict(X)

df_tmp.plot(y=["predicted", "value"])

## Závěr
* lineární regrese nám vytváří model, na základě kterého můžeme poté předvídat "budoucnost"
* model může fungovat pouze na datech (rozsazích), na jakých natrénován
* regrese není pouze prokládání funkcí pro lepší vizualizaci